In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 120.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.6 MB/s eta 0:00:00


In [ ]:
!pip install pyngrok

In [ ]:
pip install streamlit-folium


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.4/328.4 kB 7.3 MB/s eta 0:00:00


In [ ]:
pip install nbconvert


In [ ]:
%%writefile app.py
import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
import streamlit as st
import plotly.graph_objects as go

# ------------------ CLASS NAMES ------------------
class_names = [
    'animal fish', 'animal fish bass', 'fish sea_food black_sea_sprat',
    'fish sea_food gilt_head_bream', 'fish sea_food hourse_mackerel',
    'fish sea_food red_mullet', 'fish sea_food red_sea_bream',
    'fish sea_food sea_bass', 'fish sea_food shrimp',
    'fish sea_food striped_red_mullet', 'fish sea_food trout'
]
# List of colors (can be customized)
colors = [
    'darkred', 'darkgreen', 'darkblue', 'darkorange', 'indigo', 'darkviolet', 'saddlebrown',
    'goldenrod', 'darkcyan', 'darkmagenta', 'darkslategray'
]

# ------------------ DEVICE SETUP ------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ------------------ LOAD ALL MODELS ------------------
def load_models():
    model_dict = {}

    # VGG16
    vgg16 = models.vgg16(pretrained=False)
    vgg16.classifier[3] = torch.nn.Linear(4096, 1024)
    vgg16.classifier[6] = torch.nn.Linear(1024, len(class_names))
    vgg16.load_state_dict(torch.load('vgg16_model.pth', map_location=device))
    model_dict['VGG16'] = vgg16.to(device).eval()

    # ResNet50
    resnet50 = models.resnet50(pretrained=False)
    resnet50.fc = torch.nn.Linear(resnet50.fc.in_features, len(class_names))
    resnet50.load_state_dict(torch.load('resnet50_model.pth', map_location=device))
    model_dict['ResNet50'] = resnet50.to(device).eval()

    # MobileNet
    mobilenet = models.mobilenet_v2(pretrained=False)
    mobilenet.classifier[1] = torch.nn.Linear(mobilenet.classifier[1].in_features, len(class_names))
    mobilenet.load_state_dict(torch.load('mobilenet_model.pth', map_location=device))
    model_dict['MobileNet'] = mobilenet.to(device).eval()

    # InceptionV3
    inception = models.inception_v3(pretrained=False, aux_logits=False)
    inception.fc = torch.nn.Linear(inception.fc.in_features, len(class_names))
    inception.load_state_dict(torch.load('inceptionv3_model.pth', map_location=device), strict=False)
    model_dict['InceptionV3'] = inception.to(device).eval()

    # EfficientNetB0
    efficientnet = models.efficientnet_b0(pretrained=False)
    efficientnet.classifier[1] = torch.nn.Linear(efficientnet.classifier[1].in_features, len(class_names))
    efficientnet.load_state_dict(torch.load('efficientnetb0_model.pth', map_location=device))
    model_dict['EfficientNetB0'] = efficientnet.to(device).eval()

    return model_dict
#To select the model
models_dict = load_models()

# ------------------ IMAGE TRANSFORM ------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# ------------------ PREDICTION FUNCTION ------------------
def predict_image_class(img_path, model_choice):
    image = Image.open(img_path).convert('RGB')
    input_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        model = models_dict[model_choice]
        output = model(input_tensor)
        softmax_output = torch.nn.functional.softmax(output, dim=1)
        _, predicted_idx = torch.max(output, 1)
        predicted_class = class_names[predicted_idx.item()]
        confidence = softmax_output[0, predicted_idx.item()].item()

    return predicted_class, confidence

# ------------------ STREAMLIT UI ------------------

# Title with orange color using markdown
st.markdown("""<h1 style="color: orange;">Fish Classification App</h1>""", unsafe_allow_html=True)

# Radio button in the sidebar for navigation
page = st.sidebar.radio("Select a page:", ["Home", "Fish Image Prediction"])


# Home page content (no video)
if page == "Home":
    # Home page Title
    st.markdown("""<h2 style="color: orange;">Welcome to the Fish Classification App!</h2>""", unsafe_allow_html=True)
    #To show the image
    st.image("/content/ChatGPT Image Apr 28, 2025, 05_26_38 PM.png")
    #subheader for home page
    st.subheader("About this App")
    #To give some inforamtion
    st.write("""
    This app helps you identify different fish species from images using deep learning models.
    Upload a fish image, select a model, and see the prediction with confidence score visualization!
    """)
    #subheader for fish classes
    st.subheader("Supported Fish Categories")

    # Display each fish class with its corresponding color
    for i, fish in enumerate(class_names):
      #markdown for fish class with different colour
      st.markdown(f"<p style='color:{colors[i]};'>{fish}</p>", unsafe_allow_html=True)
    #subheader for homepage
    st.subheader("Key Features")
    #information of page
    st.markdown("""""<h4 style="color: orange;">
    - Upload an image of a fish and predict its category
    - Choose among 5 powerful models: VGG16, ResNet50, MobileNet, InceptionV3, EfficientNetB0
    - Visualize prediction confidence with Progress Bar and Gauge Meter
    - Fast and efficient prediction
    - Clean, simple, and intuitive user interface</h4> """)
    #information
    st.info("👉 Use the sidebar to switch to 'Fish Image Prediction' and try it out!")
    #Agian subheader
    st.subheader("Ready to Classify?")


# Fish Image Prediction page content
elif page == "Fish Image Prediction":
    st.header("Fish Image Prediction")

    uploaded_image = st.sidebar.file_uploader("Upload an image of a fish", type=["jpg", "jpeg", "png"])
    model_choice = st.sidebar.selectbox(
        "Select a Pre-trained Model",
        ["VGG16", "ResNet50", "MobileNet", "InceptionV3", "EfficientNetB0"]
    )
    #To create submit button
    submit_button = st.sidebar.button("Submit")

    #upload image
    if uploaded_image is not None and submit_button:
        image = Image.open(uploaded_image)
        st.image(image, caption="Uploaded Image", use_container_width=True)

       #spinner model choice and confideance socore
        with st.spinner('Predicting...'):
            predicted_class, confidence = predict_image_class(uploaded_image, model_choice)

        #predicted class
        st.success(f"Predicted Species: {predicted_class}")
        #metric score of the class
        st.metric("Confidence", f"{confidence*100:.2f}%")
        #confidence socore
        st.progress(confidence)
        #gauge meter with number
        fig = go.Figure(go.Indicator(
            mode="gauge+number",
            value=confidence * 100,
            title={'text': "Model Confidence"},
            gauge={
                'axis': {'range': [0, 100]},
                'bar': {'color': "blue"},
                'steps': [
                    {'range': [0, 50], 'color': "red"},
                    {'range': [50, 80], 'color': "yellow"},
                    {'range': [80, 100], 'color': "green"}
                ]
            }
        ))
        st.plotly_chart(fig, use_container_width=True)


Overwriting app.py


In [ ]:
!npm install localtunnel

⠙⠹⠸⠼
up to date, audited 23 packages in 602ms
⠼
⠼3 packages are looking for funding
⠼  run `npm fund` for details
⠼
2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠼

In [ ]:
!streamlit run /content/app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.125.159.139
⠙your url is: https://young-apples-cheer.loca.lt
